In [ ]:
%pip install pandas
%pip install numpy

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# read all data
ph = pd.read_csv('data/share-of-deaths-homicides.csv') # homicides
emis = pd.read_csv('data/GCB2022v27_MtCO2_flat.csv') # emissions
gdp = pd.read_csv('data/gdp.csv') # gdp
pop = pd.read_csv('data/population.csv') # population
ener = pd.read_csv('data/energy.csv') # energy


In [ ]:
# standardize the column names
ph.rename(columns={'Entity' : 'Country'}, inplace=True)
ph.rename(columns={'Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Percent)' : 'Homicide Rate'}, inplace=True)

gdp.rename(columns={'Country Name' : 'Country'}, inplace=True)

pop.rename(columns={'Country Name' : 'Country'}, inplace=True) 

emis.rename(columns={'Total' : 'Total Emissions'}, inplace=True)
emis.rename(columns={'Per Capita' : 'Emissions Per Capita'}, inplace=True)

ener.rename(columns={'Energy_consumption' : 'Energy Consumption'}, inplace=True)
ener.rename(columns={'Energy_production' : 'Energy Production'}, inplace=True)

In [ ]:
# drop unwanted columns
emis = emis.drop(columns = ['Other'])
emis = emis.drop(columns=['ISO 3166-1 alpha-3'])

ph = ph.drop(columns=['Code'])

gdp = gdp.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code'])

ener = ener.drop(columns=['GDP', 'Population', 'Energy_intensity_per_capita', 'Energy_intensity_by_GDP', 'CO2_emission'])

In [ ]:
# debucketizing gdp
melted_gdp = gdp.melt(id_vars=['Country'], var_name="Year", value_name='GDP per capita')
melted_gdp = melted_gdp.sort_values(["Country","Year"])
melted_gdp = melted_gdp.reset_index(drop=True)
melted_gdp['Year'] = melted_gdp['Year'].astype(int)
melted_gdp['GDP per capita'] = melted_gdp['GDP per capita'].replace(np.nan,0)

# debucketizing population
melted_pop = pop.melt(id_vars=['Country'], var_name="Year", value_name='Population')
melted_pop = melted_pop.sort_values(["Country","Year"])
melted_pop = melted_pop.reset_index(drop=True)
melted_pop['Year'] = melted_pop['Year'].astype(int)
melted_pop['Population'] = melted_pop['Population'].replace(np.nan,0)



In [ ]:
# drop rows with year < 1990 and year > 2019
emis = emis[emis['Year'] >= 1990]
emis = emis[emis['Year'] <= 2019]

gdp = melted_gdp[melted_gdp['Year'] >= 1990]
gdp = melted_gdp[melted_gdp['Year'] <= 2019]

pop = melted_pop[melted_pop['Year'] >= 1990]
pop = melted_pop[melted_pop['Year'] <= 2019]

ener = ener[ener['Year'] >= 1990]
ener = ener[ener['Year'] <= 2019]

In [ ]:
# remove NaN values
emis.isnull().sum()

c = emis.groupby('Country')

c.apply(lambda x: x.isnull().sum())


In [ ]:
# set year column as index
emis.set_index('Year', inplace=True)

# fill missing values with previous and forward 5-year averages
emis.fillna(emis.rolling(window=11, min_periods=1).mean().shift(1).fillna(method='bfill'), inplace=True)

# reset index
emis.reset_index(inplace=True)

emis.isnull().sum()

In [ ]:
# replacing nan values with 0
ener['Energy Consumption'] = ener['Energy Consumption'].replace(np.nan,0)
ener['Energy Production'] = ener['Energy Production'].replace(np.nan,0)

# standardize row values under energy type and country
ener.replace('all_energy_types', 'Total', inplace=True)
ener.replace('coal', 'Coal', inplace=True)
ener.replace('petroleum_n_other_liquids', 'Oil', inplace=True)
ener.replace('natural_gas', 'Gas', inplace=True)
ener.replace('renewables_n_other', 'Other', inplace=True)
ener.replace('World', 'Global', inplace=True)

# pivot the energy table
ener_pivot = ener.pivot_table(index=['Country', 'Year'], columns='Energy_type', values=['Energy Consumption', 'Energy Production'])

# flatten the column names
ener_pivot.columns = [' '.join(col).strip() for col in ener_pivot.columns.values]

# drop nuclear energy (not relevant to our selected data)
ener_pivot['Energy Production Total'] = ener_pivot['Energy Production Total'] - ener_pivot['Energy Production nuclear']
ener_pivot['Energy Consumption Total'] = ener_pivot['Energy Consumption Total'] - ener_pivot['Energy Consumption nuclear']
ener_pivot = ener_pivot.drop(columns=['Energy Consumption nuclear', 'Energy Production nuclear'])

ener_pivot[:10]

In [ ]:
# merging tables into one table for exporting
merged = pd.merge(ph, emis, how='inner', on=['Country', 'Year'])
merged = pd.merge(merged, gdp, how="left", on=['Country','Year'])
merged = pd.merge(merged, pop, how='left', on=['Country','Year'])
merged = pd.merge(merged, ener_pivot, how='left', on=['Country','Year'])


In [ ]:
# checking for duplicates
duplicates = merged.duplicated(subset=['Country', 'Year'], keep=False)

# print resulting dataframe
# if there are no duplicates, the dataframe will be empty
print(merged[duplicates])

In [ ]:
merged.describe()

In [ ]:
#Generating measures
merged['Homicide per capita'] = (merged['Homicide Rate'] / merged['Population']) * 100000
merged['Homicide per emissions'] = merged['Homicide Rate'] / merged['Total Emissions']
merged['GDP'] = merged['GDP per capita'] * merged['Population']
merged['Energy Production to Consumption'] = merged['Energy Production Total'] / merged['Energy Consumption Total']

In [ ]:
merged[:25]

In [ ]:
# replacing any missing values / infinities with 0
merged = merged.replace(np.nan, 0)
merged = merged.fillna(0)
merged['Homicide per emissions'].replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
merged.sum()

In [ ]:
merged.dtypes

In [ ]:
# exporting all data to csv
merged.to_csv("~/Desktop/datascience/phase2/CSI4142-Project/out.csv", index=False)